In [1]:
import json
import numpy as np
import pandas
from pprint import pprint
from datetime import datetime, timedelta
pandas.options.display.max_colwidth = -1
print('ok')

ok


In [20]:
allgames = []
#yeararray = [1921, 1922, 1925, 1927, 1928]
#for i in np.linspace(1930,2016,87):
#    yeararray.append(i)
yeararray = [2016]
    
years_with_derived = []
for i in np.linspace(1942,1959,18):
    years_with_derived.append(i)
for i in np.linspace(1962,1968,7):
    years_with_derived.append(i)
for i in np.linspace(1971,1973,3):
    years_with_derived.append(i)
print('years identified')
    
for thisyear in yeararray:
#    print('Starting to parse the year {0:.0f}'.format(thisyear))
    filename = 'raw/events_raw_{0:.0f}.json'.format(thisyear)
    print(filename)
    with open(filename,'r') as f:
        games = json.load(f)
    if (thisyear in years_with_derived):
        derived_filename = 'with_derived/events_with_derived_{0:.0f}.json'.format(thisyear)
        print(derived_filename)
        with open(derived_filename,'r') as f:
            games_derived = json.load(f)
        games = games + games_derived
    allgames = allgames + games

#    games_df = pandas.DataFrame(games)
#    games_df = games_df.set_index('id')
    print('{0:,.0f} games found'.format(len(allgames)))

    allgames_df = pandas.DataFrame()
    for i in range(0, len(allgames)):
        if ((np.mod(i, 100) == 0) | (i==len(allgames)-1)):
            print('Reading game number {0:,.0f}'.format(i))
        game_df = pandas.DataFrame.from_records(allgames[i]['info'], index=[allgames[i]['id']])
        allgames_df = pandas.concat((allgames_df, game_df))
    allgames_df.index.name = 'gameID'

    allgames_df['attendance'] = pandas.to_numeric(allgames_df['attendance'], errors='coerce')
    allgames_df['date'] = pandas.to_datetime(allgames_df['date'], format='%Y/%m/%d', errors='coerce')
    allgames_df['number'] = pandas.to_numeric(allgames_df['number'], errors='coerce')
    allgames_df['starttime'] = pandas.to_datetime(allgames_df['starttime'], format='%I:%M%p', errors='coerce')
    allgames_df['temp'] = pandas.to_numeric(allgames_df['temp'], errors='coerce')
    allgames_df['timeofgame'] = pandas.to_numeric(allgames_df['timeofgame'], errors='coerce').dropna().apply(lambda x: timedelta(minutes=x))
    allgames_df['windspeed'] = pandas.to_numeric(allgames_df['windspeed'], errors='coerce')
    print('finished reading data')
    #allgames_df.tail(3).transpose()
    allgames_df.to_csv('games/gameinfo_from_events_{0:.0f}.csv'.format(thisyear))
    print('output written')

years identified
raw/events_raw_2016.json
2,428 games found
Reading game number 0
Reading game number 100
Reading game number 200
Reading game number 300
Reading game number 400
Reading game number 500
Reading game number 600
Reading game number 700
Reading game number 800
Reading game number 900
Reading game number 1,000
Reading game number 1,100
Reading game number 1,200
Reading game number 1,300
Reading game number 1,400
Reading game number 1,500
Reading game number 1,600
Reading game number 1,700
Reading game number 1,800
Reading game number 1,900
Reading game number 2,000
Reading game number 2,100
Reading game number 2,200
Reading game number 2,300
Reading game number 2,400
Reading game number 2,427
finished reading data
output written


In [ ]:
# runs in about 5 minutes
with open('events_with_derived.json','r') as f:
    allgames = json.load(f)

allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')

print('done')

In [ ]:
info_df = pandas.DataFrame.from_records(allgames_df['info'], index=allgames_df.index)

column_list = ['visteam', 'hometeam', 'date', 'number', 'starttime', 'daynight']
column_list += ['site', 'timeofgame', 'attendance']
column_list += ['temp', 'fieldcond', 'precip', 'sky', 'winddir', 'windspeed']
column_list += ['umphome', 'ump1b', 'umb2b', 'umb3b', 'umplf', 'umprf']
column_list += ['wp', 'lp', 'save']
column_list += ['how_scored', 'oscorer', 'pitches', 'usedh']

#info_df.columns = column_list

#number -> gameno
#timeofgame -> length

info_df['attendance'] = pandas.to_numeric(info_df['attendance'], errors='coerce')
info_df['date'] = pandas.to_datetime(info_df['date'], errors='coerce')
info_df['number'] = pandas.to_numeric(info_df['number'], errors='coerce')
info_df['starttime'] = pandas.to_datetime(info_df['starttime'], format='%l%M%P', errors='coerce')
info_df['temp'] = pandas.to_numeric(info_df['temp'], errors='coerce')
info_df['timeofgame'] = pandas.to_timedelta(info_df['timeofgame'].astype(str) + ' minutes', unit='m', errors='coerce')
info_df = info_df.fillna('')

info_df.to_csv('gameinfo_from_events.csv')
print('Done')